# Apple Health convert XML to CSV

## Setup

### Import sub-modules

In [18]:
# python
import sys
import os
import datetime
import enum

# iPython
import IPython
from IPython.display import display
from IPython.display import Image

# pandas
import pandas as pd

# numpy
import numpy as np

# plotly
import plotly as ply
import plotly.figure_factory as ff
ply.offline.init_notebook_mode(connected=True)
import plotly.io as pio

# Apple Health Data Extractor
# https://github.com/markwk/qs_ledger/tree/master/apple_health
import HealthDataExtractor

# watermark
import watermark
%load_ext watermark
%watermark -a "Silvan Zahno" -d -v -iv -m -h

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
numpy               1.16.2
IPython             7.4.0
pandas              0.24.2
plotly              3.8.1
watermark           1.8.1
HealthDataExtractor 1.3
Silvan Zahno 2019-07-02 

CPython 3.7.3
IPython 7.4.0

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
CPU cores  : 8
interpreter: 64bit
host name  : WE6996


### Configuration

In [19]:
# Setup local input directories
inputDir = "in/"

if (os.path.exists(inputDir)) is False:
    os.makedirs(inputDir)
if (os.path.isdir(inputDir)) is False:
    raise NotADirectoryError("{} is not a directory".format(inputDir))

In [20]:
# Setup local input directories
outputDir = "out/"

if (os.path.exists(outputDir)) is False:
    os.makedirs(outputDir)
if (os.path.isdir(outputDir)) is False:
    raise NotADirectoryError("{} is not a directory".format(outputDir))

In [21]:
# Graph output Options
class GraphOutputOption(enum.Enum):
    none = 'none'                     # Do not generate any plots
    inline = 'inline'                 # Generate inline plots only
    htmlFile = 'extFile'              # Generate plots in external files (html or png or ...)
    both = 'both'                     # Generate all plots inline and external
    
class GraphInteractionOption(enum.Enum):
    static = 'static'                 # Generate static inline plots (as images)
    interactive = 'interactive'       # Generate interactive inline plots

notebookGraphingInteraction = GraphInteractionOption('interactive')
notebookGraphingOutputs = GraphOutputOption('both')

ext_file = ".svg"

staticImageSize = {'width':1000, 'height':500, 'scale':1}

GraphAutoOpenHTML = False              # Auto open external HTML files [True/False]

class PrintOutputOption(enum.Enum):
    none = 'none'                     # Do not output either to file or console
    console = 'console'               # Send to console
    file = 'file'                     # Send to file
    both = 'both'                     # Send to console and file


In [22]:
# Pandas output options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 100)

### Common functions

In [23]:
# Plot figures
def plot_figure(graphFilename, fig, overrideNotebookGraphingInteraction=None, height=350):
    """Creates the graph plots depending on the choosen option

    Args:
        graphFilename: Filename of the output html-file in case of Option htmlfile
        fig: ply.graph_objs.Figure(data=traces, layout=mylayout)
        overrideNotebookGraphingInteraction: overrides graphic output for this figure
        height: height of the static image export
    Returns:
        None
    Raises:
        None

    """
    if (notebookGraphingOutputs == GraphOutputOption('extFile')) or (notebookGraphingOutputs == GraphOutputOption('both')):
        if (os.path.splitext(graphFilename)[1] == '.png' or 
           os.path.splitext(graphFilename)[1] == '.jpg' or 
           os.path.splitext(graphFilename)[1] == '.svg' or 
           os.path.splitext(graphFilename)[1] == '.pdf'):
            pio.write_image(fig, graphFilename, height=height)
        elif os.path.splitext(graphFilename)[1] == '.html':
            ply.offline.plot(fig, filename=graphFilename, auto_open=GraphAutoOpenHTML)
    if (notebookGraphingOutputs == GraphOutputOption('inline')) or (notebookGraphingOutputs == GraphOutputOption('both')):
        choice = None
        if overrideNotebookGraphingInteraction == None:
            if notebookGraphingInteraction == GraphInteractionOption('static'):
                choice = 'static'
            elif notebookGraphingInteraction == GraphInteractionOption('interactive'):
                choice = 'interactive'
        elif overrideNotebookGraphingInteraction == GraphInteractionOption('static'):
            choice = 'static'
        elif overrideNotebookGraphingInteraction == GraphInteractionOption('interactive'):
            choice = 'interactive'
        else:
            choice = 'static'
            
        if choice == 'static':
            plot_img = pio.to_image(fig, format='png', width=staticImageSize['width'], height=staticImageSize['height'], scale=staticImageSize['scale'])
            display(Image(plot_img))
        else:
            ply.offline.iplot(fig)
            

In [24]:
def print_file(text="", file=None, outputOption=PrintOutputOption('console'), append=True):
    """Custom print function to print to console and/or file

    Args:
        text: String to be used
        file: output file path for fileoutput
        fileoutput: bool sends string to file
        consoleoutput: PrintOutputOption to specify where to send the string
        append: For fileoutput only, create new file or append to existing
    Returns:
        None
    Raises:
        None

    """
    if (outputOption == PrintOutputOption('file') or outputOption == PrintOutputOption('both')):
        if append:
            with open(file, "a+") as file:
                file.write(text+"\n")#os.linesep)
        else:
            with open(file, "w+") as file:
                file.write(text+"\n")#os.linesep)
    if (outputOption == PrintOutputOption('console') or outputOption == PrintOutputOption('both')):
        print(text)

## Data Import

In [25]:
dirlist = os.listdir(inputDir)
print("Files from directory ", inputDir)
for file in dirlist:
  print("  * ",file)


Files from directory  in/
  *  export.xml
  *  export_cda.xml


### Convert to CSV

In [27]:
data = HealthDataExtractor.HealthDataExtractor(inputDir + "export.xml")
data.report_stats()
data.extract()

Reading data from in/export.xml . . . done
Unexpected node of type ExportDate.

Tags:
ExportDate: 1
Me: 1
Record: 418910
Workout: 110

Fields:
HKCharacteristicTypeIdentifierBiologicalSex: 1
HKCharacteristicTypeIdentifierBloodType: 1
HKCharacteristicTypeIdentifierDateOfBirth: 1
HKCharacteristicTypeIdentifierFitzpatrickSkinType: 1
creationDate: 419020
device: 149610
duration: 110
durationUnit: 110
endDate: 419020
sourceName: 419020
sourceVersion: 150033
startDate: 419020
totalDistance: 110
totalDistanceUnit: 110
totalEnergyBurned: 110
totalEnergyBurnedUnit: 110
type: 418910
unit: 418318
value: 418911
workoutActivityType: 110

Record types:
ActiveEnergyBurned: 110
BloodAlcoholContent: 4
BodyFatPercentage: 2
BodyMass: 21
BodyMassIndex: 20
DietaryCaffeine: 4
DietaryWater: 18
DistanceCycling: 17
DistanceWalkingRunning: 185915
FlightsClimbed: 52615
HeartRate: 56
Height: 1
SleepAnalysis: 592
StepCount: 179535

Opening C:\Users\silvan.zahno\Workspace\Dropbox\Programming\ipython\01_Mine\apple_he